In [ ]:
import sympy as sp

init = np.array([0.3, 0.5, 0.15, 0.1, 0.1])
b0, b1 = 1., 1.
d1, d2, d3 = 0.2, 0.73, 0.338
e2, e3, e4 = 0.3, 0.2, 0.12
h1, p2, n4 = 0.05, 0.05, 0.05
z0, f1, s2 = 0.1, 0.1, 0.048
alpha = 0.25
T = 100.

DC, PP, PC, SC, TC = sp.symbols('DC PP PC SC TC', real=True)

state = (DC, PP, PC, SC, TC)

params_dict = {
    "b0": 1.,
    "b1": 1.,
    "d1": 0.2,
    "d2": 0.73,
    "d3": 0.338,
    "e2": 0.3,
    "e3": 0.2,
    "e4": 0.12,
    "h1": 0.0,
    "p2": 0.0,
    "n4": 0.05,
    "z0": 0.1,
    "f1": 0.1,
    "s2": 0.048,
    "a1": 1.
    }

params = {name: sp.symbols(name) for name in params_dict.keys()}

b0, b1, a1 = params["b0"], params["b1"], params["a1"]
d1, d2, d3 = params["d1"], params["d2"], params["d3"]
e2, e3, e4 = params["e2"], params["e3"], params["e4"]
h1, p2, n4 = params["h1"], params["p2"], params["n4"]
z0, f1, s2 = params["z0"], params["f1"], params["s2"]

dDC_dt = DC * ( z0 * (d1*PP + d2*PC + d3*SC + n4*TC) ) * (1 - b1 * DC)
dPP_dt = PP * ( a1 * (1 - b1* PP) + f1*DC - (d1 - s2)*PC - h1 )
dPC_dt = PC * ( e2*PP - d2*SC - p2 )
dSC_dt = SC * ( e3*PC - d3*TC )
dTC_dt = TC * ( e4*SC - n4 )

odes = (dDC_dt, dPP_dt, dPC_dt, dSC_dt, dTC_dt)

# build dX/dt = 0
eqns = [sp.Eq(expr, 0) for expr in odes]

# put numbers in parameters
eqns_num = [eq.subs({params[k]: v for k, v in params_dict.items()}) for eq in eqns]

# symbolic solver
symbolic_solutions = sp.solve(eqns_num, state, dict=True, simplify=True)
print(f"Symbolic solve returned {len(symbolic_solutions)} candidate solutions.")
print("---- symbolic solutions (raw) ----")
for sol in symbolic_solutions:
    print(sol)

In [ ]:
# Jacobian solver
J = sp.Matrix(odes).jacobian(state)
print("Symbolic Jacobian:")
display(J)

In [ ]:
# Numeric Jacobian at equilibrium
eq_pt = symbolic_solutions[-1]
eq_pt

# evaluate Jacobian numerically
subs_dict = {params[k]: v for k, v in params_dict.items()}
subs_dict.update(eq_pt)

# numeric Jacobian
J_numeric = np.array(J.subs(subs_dict).evalf(), dtype=float)
print("Numeric Jacobian at the chosen equilibrium:\n", J_numeric)

# eigenvalues
eigvals = np.linalg.eigvals(J_numeric)
print("\nEigenvalues of the Jacobian:")
for ev in eigvals:
    print(f"{ev: .6f}")